In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import json
from datetime import datetime

# Advanced ML imports
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
import shap
import joblib
import pickle

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Statistical analysis
import scipy.stats as stats
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)


In [2]:
# 1. SETUP AND DATA LOADING  

print("="*80)
print("ADVANCED MODEL BUILDING AND EVALUATION")
print("="*80)

# Load pre-trained models and data
print("Loading pre-trained models and data...")

try:
    best_export_model = joblib.load('best_export_model.pkl')
    best_class_model = joblib.load('best_classification_model.pkl')
    export_scaler = joblib.load('export_scaler.pkl')
    class_scaler = joblib.load('classification_scaler.pkl')
    
    # Load training data
    df = pd.read_csv('world_trade_cleaned.csv')
    country_df = df[df['Is_Country']].copy()
    
    # Load feature lists
    with open('export_features.txt', 'r') as f:
        export_features = [line.strip() for line in f.readlines()]
    
    with open('classification_features.txt', 'r') as f:
        class_features = [line.strip() for line in f.readlines()]
    
    print("✓ Models and data loaded successfully")
    
except Exception as e:
    print(f"Error loading saved models: {e}")
    print("Please run Notebook 03 first to train models")
    raise


ADVANCED MODEL BUILDING AND EVALUATION
Loading pre-trained models and data...
Error loading saved models: [Errno 2] No such file or directory: 'best_export_model.pkl'
Please run Notebook 03 first to train models


FileNotFoundError: [Errno 2] No such file or directory: 'best_export_model.pkl'